<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/wordmark_black65ee464.png" width="700">

# Week 2 : Final Challenge

**Welcome to the final challenge!**  

In the previous notebook we've seen how we can use the VQC class in Aqua to classify the digits `0` and `1`. However, classifying `0` and `1` is relatively simple as digits `0` and `1` are easily distinguishable. `4` and `9` however, are notoriously similar, with a _loop_ on the top and a _line_ on the bottom. This can be corroborated looking at our 2-D t-SNE plot from the previous notebook (Fig.2), we see that `0` and `1` are clustered relatively far from each other making them easily distinguishable, however `4` and `9` are overlapping. In this challenge we are providing you with a dataset with digits reduced to **dimension 3**. For example, in Fig.1 we can see the dimension reduction of the 784 dimension vector for digit `4` into a dimension 3 feature vector. 

**Fig.1 : Features of the digit `4` after reducing dimension to 3:** 
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/four2a7701f.png" width="700">

**Fig.2 : MNIST dataset after dimension reduction to 2 as given in the previous notebook:**
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/mnist_plot53adb39.png" width="400">

## Challenge Question   
Use the VQC method from Aqua to classify the digits `4` and `9` as given in the dataset **challenge_dataset_4_9_dim3_training.csv** provided to you. 

## Rules and Guidelines

* Your `QuantumCircuit` can have a **maximum of 6 qubits**.
* **Cost of the circuit should be less than 2000**.  
* You should not change names of the functions `feature_map()` , `variational_circuit()`  and `return_optimal_params()`.
* All the functions must return the value types as mentioned. 
* All circuits must be Qiskit generated.
* Best of all submissions is considered for grading.

## Judging criteria 

* Primary judgement is based on the **accuracy of the model**, higher the better. **Accuracies which differ by less than 0.005 will be considered to be equal**. ex: Accuracies 0.7783 and 0.7741 will be considered to be equal.
* If the accuracies are tied, the tie will be broken using **cost of the circuit** as the metric, lower the better. 
* In the case that both accuracy of the model and cost of the circuit are equal, **time of submission** is taken into account, Earlier the better. 

_**Important Note:**_ The **leaderboard shown during the progress of the competition** will only display accuracy of the model and is **not the final leaderboard**. Breaking ties between accuracy of the model by considering lower **cost of circuit** will only be done after the competition ends. **The final leaderboard will be announced post the event** which will take into consideration cost of the circuit and time of submission. 

## Certificate Eligibility

Everyone who scores an **accuracy greater than 0.70 (i.e, 70%) will be eligible for a certificate**. 


An explanation on how to calculate the accuracy of the model and the cost of the circuit is given in the end inside the `grade()` function. Before you submit, make sure the grading function is running on your device. To save time you can also use the grading function provided to calculate the accuracy and circuit cost without having to submit your solution onto HackerEarth. Remember, your final score will be determined using the same grading methods as given in this notebook, but will be evaluated on unseen datapoints.

In [1]:
# installing a few dependencies
# !pip install --upgrade seaborn==0.10.1
# !pip install --upgrade scikit-learn==0.23.1
# !pip install --upgrade matplotlib==3.2.0
# !pip install --upgrade pandas==1.0.4
# !pip install --upgrade qiskit==0.19.6 
# !pip install --upgrade plotly==4.9.0

# the output will be cleared after installation
from IPython.display import clear_output
clear_output()

In [2]:
# we have imported a few libraries we thing might be useful 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer

from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA


# The the write_and_run() magic function creates a file with the content inside the cell that it is run. 
# You have used this in previous exercises for creating your submission files. 
# It will be used for the same purpose here.

from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Solution

## Data loading 

This notebook has helper functions and code snippets to save your time and help you concentrate on what's important: Increasing the accuracy of your model. Running the cell below will import the challenge dataset and will be available to you as `data`. Before running the cell below store the dataset in this file structure (or change the `data_path` accordingly):  

- `challenge_notebook.ipynb`
- `dataset`
    - `challenge_dataset_4_9.csv`


In [3]:
#data_path='./dataset/'
data_path='../'
data = np.loadtxt(data_path + "challenge_dataset_4_9_dim3_training.csv", delimiter=",")

# extracting the first column which contains the labels
data_labels = data[:, :1].reshape(data.shape[0],)
# extracting all the columns but the first which are our features
data_features = data[:, 1:]

## Visualizing the dataset

Before we dive into solving the question it is always beneficial to look at the dataset pictographically. This will help us understand patterns which we could leverage when designing our feature maps and variational circuits for example.

## Extracting the training dataset

The given dataset has already been reduced in dimension and normalized, so, further pre-processing isn't techincally required. You can do so if you want to, but the testing dataset will be of the same dimension and normalisation as the training dataset provided. Training a dataset of size 6,000 will take multiple hours so you'll need to extract a subset of the dataset to use as a training dataset. The accuracy of the model may vary based on the datapoints and size of the training dataset you choose. Thus, experimenting with various sizes and datapoints will be necessary. For example, Increasing the training dataset size may increase the accuracy of the model however it will increase the training time as well.

Use the space below to extract your training dataset from `data`. For your convenience `data` has been segregated into `data_labels` and `data_features`.

* `data_labels` : 6,000 $\times$ 1 column vector with each entry either `4` or `9` 
* `data_features` : 6,000 $\times$ 3 matrix with each row having the feature corresponding to the label in `data_labels`

**Note:** This process was done in the previous [VQC notebook](https://github.com/Qiskit-Challenge-India/2020/blob/master/Day%206%2C%207%2C8/VQC_notebook.ipynb) with `0` and `1` labels and can be modified and used here as well. 

In [13]:
### WRITE YOUR CODE BETWEEN THESE LINES - START

# do your classical pre-processing here

# store your training and testing datasets to be input in the VQC optimizer in the "training_input" and 
# "testing_input" variables respectively. These variables will eb accessed whiile creating a VQC instance later. 
four_datapoints = []
nine_datapoints = []

for i in range(6000):
    if data_labels[i] == 4:
        four_datapoints.append(data_features[i])

for i in range(6000):
    if data_labels[i] == 9:
        nine_datapoints.append(data_features[i])
        
four_datapoints = np.array(four_datapoints)
nine_datapoints = np.array(nine_datapoints)
np.random.shuffle(four_datapoints)
np.random.shuffle(nine_datapoints)

train_num = 100
test_num = 100

training_input = {4:four_datapoints[:train_num], 9:nine_datapoints[:train_num]}
test_input = {4:four_datapoints[-test_num:], 9:nine_datapoints[-test_num:]}

### WRITE YOUR CODE BETWEEN THESE LINES - END

## Building a Quantum Feature Map

Given below is the `feature_map()` function. It takes no input and has to return a feature map which is either a `FeatureMap` or `QuantumCircuit` object. In the previous notebook you've learnt how feature maps work and the process of using existing feature maps in Qiskit or creating your own. In the space given **inside the function** you have to create a feature map and return it.   


**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `feature_map.py`

In [5]:
%%write_and_run feature_map.py

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

def feature_map(): 

    ckt9_depth=3
    num_qubits=6
    x = ParameterVector('x', length=3)
    feature_map=QuantumCircuit(num_qubits)
    
    for i in range(ckt9_depth):
        for j in range(num_qubits):
            feature_map.h(j)
        for j in range(2):
            feature_map.crz((np.pi-x[j])*(np.pi-x[j+1]),j,j+1)
        for j in range(2):
            feature_map.crz((np.pi-x[j]*x[j])*(np.pi-x[j+1]*x[j+1]),j+3,j+4)
        for j in range(3):
            feature_map.rx(x[j],j)
        for j in range(3):
            feature_map.rx(x[j]*x[j],j+3)

    #return the feature map which is either a FeatureMap or QuantumCircuit object
    return feature_map

In [6]:
feature_map().draw()

┌───┐                                                                 »
q_0: ┤ H ├────────────────────────────────■────────────────────────────────»
     ├───┤   ┌────────────────────────────┴────────────────────────────┐   »
q_1: ┤ H ├───┤ RZ((3.14159265358979 - x[0])*(3.14159265358979 - x[1])) ├───»
     ├───┤   └─────────────────────────────────────────────────────────┘   »
q_2: ┤ H ├─────────────────────────────────────────────────────────────────»
     ├───┤                                                                 »
q_3: ┤ H ├────────────────────────────────■────────────────────────────────»
     ├───┤┌───────────────────────────────┴───────────────────────────────┐»
q_4: ┤ H ├┤ RZ((3.14159265358979 - x[0]**2)*(3.14159265358979 - x[1]**2)) ├»
     ├───┤└───────────────────────────────────────────────────────────────┘»
q_5: ┤ H ├─────────────────────────────────────────────────────────────────»
     └───┘                                                                 »
«                                ┌──────────┐                          »
«q_0: ───────────────────────────┤ RX(x[0]) ├──────────────────────────»
«                                └──────────┘                          »
«q_1: ────────────────────────────────■────────────────────────────────»
«        ┌────────────────────────────┴────────────────────────────┐   »
«q_2: ───┤ RZ((3.14159265358979 - x[1])*(3.14159265358979 - x[2])) ├───»
«        └─────────────────────┬─────────────┬─────────────────────┘   »
«q_3: ─────────────────────────┤ RX(x[0]**2) ├─────────────────────────»
«                              └─────────────┘                         »
«q_4: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_5: ┤ RZ((3.14159265358979 - x[1]**2)*(3.14159265358979 - x[2]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«          ┌───┐          »
«q_0: ─────┤ H ├──────────»
«       ┌──┴───┴───┐ ┌───┐»
«q_1: ──┤ RX(x[1]) ├─┤ H ├»
«       ├──────────┤ ├───┤»
«q_2: ──┤ RX(x[2]) ├─┤ H ├»
«       └──┬───┬───┘ └───┘»
«q_3: ─────┤ H ├──────────»
«     ┌────┴───┴────┐┌───┐»
«q_4: ┤ RX(x[1]**2) ├┤ H ├»
«     ├─────────────┤├───┤»
«q_5: ┤ RX(x[2]**2) ├┤ H ├»
«     └─────────────┘└───┘»
«                                                                      »
«q_0: ────────────────────────────────■────────────────────────────────»
«        ┌────────────────────────────┴────────────────────────────┐   »
«q_1: ───┤ RZ((3.14159265358979 - x[0])*(3.14159265358979 - x[1])) ├───»
«        └─────────────────────────────────────────────────────────┘   »
«q_2: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«q_3: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_4: ┤ RZ((3.14159265358979 - x[0]**2)*(3.14159265358979 - x[1]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«q_5: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«                                ┌──────────┐                          »
«q_0: ───────────────────────────┤ RX(x[0]) ├──────────────────────────»
«                                └──────────┘                          »
«q_1: ────────────────────────────────■────────────────────────────────»
«        ┌────────────────────────────┴────────────────────────────┐   »
«q_2: ───┤ RZ((3.14159265358979 - x[1])*(3.14159265358979 - x[2])) ├───»
«        └─────────────────────┬─────────────┬─────────────────────┘   »
«q_3: ─────────────────────────┤ RX(x[0]**2) ├─────────────────────────»
«                              └─────────────┘                         »
«q_4: ────────────────────────────────■────────────────────────────────»
«     ┌─────

## Building a Variational Circuit

Given below is the `variational_circuit()` function. It takes no input and has to return a variational circuit which is either a `VariationalForm` or `QuantumCircuit` object. In the previous notebook you've learnt how variational circuits work and the process of using existing variational circuit in Qiskit or creating your own. You have to create a variational circuit in the space given **inside the function** and return it. You can find various variational circuits in the [Qiskit Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html) under N-local circuits.

**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `variational_circuit.py`

In [7]:
%%write_and_run variational_circuit.py

def variational_circuit():

    ckt5_depth=4
    num_qubits=3
    y = ParameterVector('y', length=num_qubits*(num_qubits+3)*ckt5_depth)
    ckt5_depth=2
    num_qubits=6
    var_circuit=QuantumCircuit(num_qubits)
    ii = 0
    
    for i in range(ckt5_depth):
        for j in range(num_qubits):
            var_circuit.rx(y[ii], j)
            ii += 1
        for j in range(num_qubits):
            var_circuit.rz(y[ii], j)
            ii += 1
        for j in range(num_qubits//2):
            for k in range(num_qubits//2):
                if j != k:
                    var_circuit.crz(y[ii], num_qubits-1-j, num_qubits-1-k)
                    ii += 1
        for j in range(num_qubits//2):
            for k in range(num_qubits//2):
                if j != k:
                    var_circuit.crz(y[ii], num_qubits-1-j-3, num_qubits-1-k-3)
                    ii += 1
        for j in range(num_qubits):
            var_circuit.rx(y[ii], j)
            ii += 1
        for j in range(num_qubits):
            var_circuit.rz(y[ii], j)
            ii += 1

    # return the variational circuit which is either a VaritionalForm or QuantumCircuit object
    return var_circuit

In [8]:
variational_circuit().draw()

┌──────────┐ ┌──────────┐             ┌───────────┐             »
q_0: ┤ RX(y[0]) ├─┤ RZ(y[6]) ├─────────────┤ RZ(y[19]) ├─────────────»
     ├──────────┤ ├──────────┤┌───────────┐└─────┬─────┘             »
q_1: ┤ RX(y[1]) ├─┤ RZ(y[7]) ├┤ RZ(y[18]) ├──────┼────────────■──────»
     ├──────────┤ ├──────────┤└─────┬─────┘      │      ┌─────┴─────┐»
q_2: ┤ RX(y[2]) ├─┤ RZ(y[8]) ├──────■────────────■──────┤ RZ(y[20]) ├»
     ├──────────┤ ├──────────┤             ┌───────────┐└───────────┘»
q_3: ┤ RX(y[3]) ├─┤ RZ(y[9]) ├─────────────┤ RZ(y[13]) ├─────────────»
     ├──────────┤┌┴──────────┤┌───────────┐└─────┬─────┘             »
q_4: ┤ RX(y[4]) ├┤ RZ(y[10]) ├┤ RZ(y[12]) ├──────┼────────────■──────»
     ├──────────┤├───────────┤└─────┬─────┘      │      ┌─────┴─────┐»
q_5: ┤ RX(y[5]) ├┤ RZ(y[11]) ├──────■────────────■──────┤ RZ(y[14]) ├»
     └──────────┘└───────────┘                          └───────────┘»
«     ┌───────────┐                          ┌───────────┐┌───────────┐»
«q_0: ┤ RZ(y[21]) ├──────■────────────■──────┤ RX(y[24]) ├┤ RZ(y[30]) ├»
«     └─────┬─────┘      │      ┌─────┴─────┐├───────────┤├───────────┤»
«q_1: ──────■────────────┼──────┤ RZ(y[23]) ├┤ RX(y[25]) ├┤ RZ(y[31]) ├»
«                  ┌─────┴─────┐├───────────┤├───────────┤├───────────┤»
«q_2: ─────────────┤ RZ(y[22]) ├┤ RX(y[26]) ├┤ RZ(y[32]) ├┤ RX(y[38]) ├»
«     ┌───────────┐└───────────┘└───────────┘├───────────┤├───────────┤»
«q_3: ┤ RZ(y[15]) ├──────■────────────■──────┤ RX(y[27]) ├┤ RZ(y[33]) ├»
«     └─────┬─────┘      │      ┌─────┴─────┐├───────────┤├───────────┤»
«q_4: ──────■────────────┼──────┤ RZ(y[17]) ├┤ RX(y[28]) ├┤ RZ(y[34]) ├»
«                  ┌─────┴─────┐├───────────┤├───────────┤├───────────┤»
«q_5: ─────────────┤ RZ(y[16]) ├┤ RX(y[29]) ├┤ RZ(y[35]) ├┤ RX(y[41]) ├»
«                  └───────────┘└───────────┘└───────────┘└───────────┘»
«     ┌───────────┐┌───────────┐             ┌───────────┐             »
«q_0: ┤ RX(y[36]) ├┤ RZ(y[42]) ├─────────────┤ RZ(y[55]) ├─────────────»
«     ├───────────┤├───────────┤┌───────────┐└─────┬─────┘             »
«q_1: ┤ RX(y[37]) ├┤ RZ(y[43]) ├┤ RZ(y[54]) ├──────┼────────────■──────»
«     ├───────────┤└───────────┘└─────┬─────┘      │      ┌─────┴─────┐»
«q_2: ┤ RZ(y[44]) ├───────────────────■────────────■──────┤ RZ(y[56]) ├»
«     ├───────────┤┌───────────┐             ┌───────────┐└───────────┘»
«q_3: ┤ RX(y[39]) ├┤ RZ(y[45]) ├─────────────┤ RZ(y[49]) ├─────────────»
«     ├───────────┤├───────────┤┌───────────┐└─────┬─────┘             »
«q_4: ┤ RX(y[40]) ├┤ RZ(y[46]) ├┤ RZ(y[48]) ├──────┼────────────■──────»
«     ├───────────┤└───────────┘└─────┬─────┘      │      ┌─────┴─────┐»
«q_5: ┤ RZ(y[47]) ├───────────────────■────────────■──────┤ RZ(y[50]) ├»
«     └───────────┘                                       └───────────┘»
«     ┌───────────┐                          ┌───────────┐┌───────────┐
«q_0: ┤ RZ(y[57]) ├──────■────────────■──────┤ RX(y[60]) ├┤ RZ(y[66]) ├
«     └─────┬─────┘      │      ┌─────┴─────┐├───────────┤├───────────┤
«q_1: ──────■────────────┼──────┤ RZ(y[59]) ├┤ RX(y[61]) ├┤ RZ(y[67]) ├
«                  ┌─────┴─────┐├───────────┤├───────────┤└───────────┘
«q_2: ─────────────┤ RZ(y[58]) ├┤ RX(y[62]) ├┤ RZ(y[68]) ├─────────────
«     ┌───────────┐└───────────┘└───────────┘├───────────┤┌───────────┐
«q_3: ┤ RZ(y[51]) ├──────■────────────■──────┤ RX(y[63]) ├┤ RZ(y[69]) ├
«     └─────┬─────┘      │      ┌─────┴─────┐├───────────┤├───────────┤
«q_4: ──────■────────────┼──────┤ RZ(y[53]) ├┤ RX(y[64]) ├┤ RZ(y[70]) ├
«                  ┌─────┴─────┐├───────────┤├───────────┤└───────────┘
«q_5: ─────────────┤ RZ(y[52]) ├┤ RX(y[65]) ├┤ RZ(y[71]) ├─────────────
«                  └───────────┘└───────────┘└───────────┘

## Choosing a Classical Optimizer

In the `classical_optimizer()` function given below you will have to import the optimizer of your choice from [`qiskit.aqua.optimizers`](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.optimizers.html) and return it. This function will not be called by the grading function `grade()` and thus the name of the function `classical_optimizer()`can be changed if needed. 

In [9]:
def classical_optimizer():
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - START
    
    # import the required clasical optimizer from qiskit.aqua.optimizers
    from qiskit.aqua.components.optimizers import SPSA
    
    # create an optimizer object
    cls_opt = COBYLA(maxiter=500, tol=0.001)
    
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - END
    return cls_opt

### Callback Function

The `VQC` class can take in a callback function to which the following parameters will be passed after every optimization cycle of the algorithm:

* `eval_count` : the evaulation counter
* `var_params` : value of parameters of the variational circuit
* `eval_val`  : current cross entropy cost 
* `index` : the batch index

In [10]:
def call_back_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("eval_val: {}".format(eval_val))
    print("index: {}".format(index))

## Optimization Step

This is where the whole VQC algorithm will come together. First we create an instance of the `VQC` class. 

In [11]:
# a fixed seed so that we get the same answer when the same input is given. 
seed = 1059

# setting our backend to qasm_simulator with the "statevector" method on. This particular setup is given as it was 
# found to perform better than most. Feel free to play around with different backend options.
backend = Aer.get_backend('qasm_simulator')
backend_options = {"method": "statevector"}

# creating a quantum instance using the backend and backend options taken before
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                   backend_options=backend_options)
#quantum_instance = QuantumInstance(backend, shots=1024, backend_options=backend_options)

# creating a VQC instance which you will be used for training. Make sure you input the correct training_dataset and 
# testing_dataset as defined in your program.
vqc = VQC(optimizer=classical_optimizer(), 
          feature_map=feature_map(), 
          var_form=variational_circuit(), 
          callback=call_back_vqc, 
          training_dataset=training_input,     # training_input must be initialized with your training dataset
          test_dataset=test_input)             # testing_input must be initialized with your testing dataset

Now, let's run the VQC classification routine

In [ ]:
start = time.process_time()

result = vqc.run(quantum_instance)

print("time taken: ")
print(time.process_time() - start)

print("testing success ratio: {}".format(result['testing_accuracy']))

eval_count: 0
var_params: [-1.84138387e+00 -3.64578861e-02 -1.85116760e+00 -1.11645741e-01
 -1.50027368e+00  2.86854642e-01  5.74410524e-01 -1.44703085e+00
 -1.03964874e+00  2.92263760e-01  4.37858456e-02 -6.13667209e-01
  1.23690658e+00  7.39538436e-02  4.30574888e-01 -9.02685620e-01
 -2.98532367e-01  6.47481102e-02 -1.14755727e+00  2.16390712e-01
  1.70598601e+00  3.39898801e-02 -5.55581699e-01 -6.28849992e-04
  3.03666491e-01 -6.62361338e-01  3.69424744e-01 -1.51452466e-01
  6.03264259e-01 -1.33908551e+00  3.12607857e-01  4.01777680e-01
 -1.36521806e+00 -5.86919950e-02 -1.65307532e+00  2.14852813e-01
  1.04549783e-01  9.43068855e-03  9.72354603e-01  1.56140977e+00
 -7.61570248e-01 -1.08747713e+00  8.87552034e-02 -9.86605215e-01
  1.64731674e+00  2.69005719e+00  5.41769493e-01  8.88944071e-01
 -1.12100278e+00  1.05309017e+00  1.52447404e+00 -9.72491188e-01
  1.47682260e-01 -1.34394590e+00 -1.74152712e-01  3.09737058e-01
 -9.04065973e-01  1.19324356e+00  3.41967992e-03  1.39936515e+00

eval_count: 7
var_params: [-8.41383875e-01 -3.64578861e-02 -1.85116760e+00  8.88354259e-01
 -1.50027368e+00  1.28685464e+00  1.57441052e+00 -1.44703085e+00
 -1.03964874e+00  2.92263760e-01  4.37858456e-02 -6.13667209e-01
  1.23690658e+00  7.39538436e-02  4.30574888e-01 -9.02685620e-01
 -2.98532367e-01  6.47481102e-02 -1.14755727e+00  2.16390712e-01
  1.70598601e+00  3.39898801e-02 -5.55581699e-01 -6.28849992e-04
  3.03666491e-01 -6.62361338e-01  3.69424744e-01 -1.51452466e-01
  6.03264259e-01 -1.33908551e+00  3.12607857e-01  4.01777680e-01
 -1.36521806e+00 -5.86919950e-02 -1.65307532e+00  2.14852813e-01
  1.04549783e-01  9.43068855e-03  9.72354603e-01  1.56140977e+00
 -7.61570248e-01 -1.08747713e+00  8.87552034e-02 -9.86605215e-01
  1.64731674e+00  2.69005719e+00  5.41769493e-01  8.88944071e-01
 -1.12100278e+00  1.05309017e+00  1.52447404e+00 -9.72491188e-01
  1.47682260e-01 -1.34394590e+00 -1.74152712e-01  3.09737058e-01
 -9.04065973e-01  1.19324356e+00  3.41967992e-03  1.39936515e+00

eval_count: 14
var_params: [-8.41383875e-01 -3.64578861e-02 -1.85116760e+00  8.88354259e-01
 -1.50027368e+00  1.28685464e+00  5.74410524e-01 -1.44703085e+00
 -1.03964874e+00  1.29226376e+00  4.37858456e-02  3.86332791e-01
  2.23690658e+00  1.07395384e+00  4.30574888e-01 -9.02685620e-01
 -2.98532367e-01  6.47481102e-02 -1.14755727e+00  2.16390712e-01
  1.70598601e+00  3.39898801e-02 -5.55581699e-01 -6.28849992e-04
  3.03666491e-01 -6.62361338e-01  3.69424744e-01 -1.51452466e-01
  6.03264259e-01 -1.33908551e+00  3.12607857e-01  4.01777680e-01
 -1.36521806e+00 -5.86919950e-02 -1.65307532e+00  2.14852813e-01
  1.04549783e-01  9.43068855e-03  9.72354603e-01  1.56140977e+00
 -7.61570248e-01 -1.08747713e+00  8.87552034e-02 -9.86605215e-01
  1.64731674e+00  2.69005719e+00  5.41769493e-01  8.88944071e-01
 -1.12100278e+00  1.05309017e+00  1.52447404e+00 -9.72491188e-01
  1.47682260e-01 -1.34394590e+00 -1.74152712e-01  3.09737058e-01
 -9.04065973e-01  1.19324356e+00  3.41967992e-03  1.39936515e+0

eval_count: 21
var_params: [-8.41383875e-01 -3.64578861e-02 -1.85116760e+00  8.88354259e-01
 -1.50027368e+00  1.28685464e+00  5.74410524e-01 -1.44703085e+00
 -1.03964874e+00  1.29226376e+00  4.37858456e-02  3.86332791e-01
  2.23690658e+00  7.39538436e-02  4.30574888e-01 -9.02685620e-01
 -2.98532367e-01  6.47481102e-02 -1.14755727e+00  2.16390712e-01
  2.70598601e+00  3.39898801e-02 -5.55581699e-01 -6.28849992e-04
  3.03666491e-01 -6.62361338e-01  3.69424744e-01 -1.51452466e-01
  6.03264259e-01 -1.33908551e+00  3.12607857e-01  4.01777680e-01
 -1.36521806e+00 -5.86919950e-02 -1.65307532e+00  2.14852813e-01
  1.04549783e-01  9.43068855e-03  9.72354603e-01  1.56140977e+00
 -7.61570248e-01 -1.08747713e+00  8.87552034e-02 -9.86605215e-01
  1.64731674e+00  2.69005719e+00  5.41769493e-01  8.88944071e-01
 -1.12100278e+00  1.05309017e+00  1.52447404e+00 -9.72491188e-01
  1.47682260e-01 -1.34394590e+00 -1.74152712e-01  3.09737058e-01
 -9.04065973e-01  1.19324356e+00  3.41967992e-03  1.39936515e+0

eval_count: 28
var_params: [-8.41383875e-01 -3.64578861e-02 -1.85116760e+00  8.88354259e-01
 -1.50027368e+00  1.28685464e+00  5.74410524e-01 -1.44703085e+00
 -1.03964874e+00  1.29226376e+00  4.37858456e-02  3.86332791e-01
  2.23690658e+00  7.39538436e-02  4.30574888e-01 -9.02685620e-01
 -2.98532367e-01  6.47481102e-02 -1.14755727e+00  2.16390712e-01
  1.70598601e+00  3.39898801e-02 -5.55581699e-01 -6.28849992e-04
  3.03666491e-01  3.37638662e-01  3.69424744e-01  8.48547534e-01
  6.03264259e-01 -1.33908551e+00  3.12607857e-01  4.01777680e-01
 -1.36521806e+00 -5.86919950e-02 -1.65307532e+00  2.14852813e-01
  1.04549783e-01  9.43068855e-03  9.72354603e-01  1.56140977e+00
 -7.61570248e-01 -1.08747713e+00  8.87552034e-02 -9.86605215e-01
  1.64731674e+00  2.69005719e+00  5.41769493e-01  8.88944071e-01
 -1.12100278e+00  1.05309017e+00  1.52447404e+00 -9.72491188e-01
  1.47682260e-01 -1.34394590e+00 -1.74152712e-01  3.09737058e-01
 -9.04065973e-01  1.19324356e+00  3.41967992e-03  1.39936515e+0

...

eval_count: 247
var_params: [-3.96043884e-01 -1.89211864e-02 -1.82597451e+00  8.77990429e-01
 -1.49550966e+00  1.27626111e+00  5.63832729e-01 -1.36722998e+00
 -7.99506360e-01  1.30454831e+00  4.53000236e-02  3.57086562e-01
  2.34173799e+00  5.36810763e-02  4.39000838e-01 -9.42271554e-01
 -3.26820875e-01  1.11048081e-01 -1.19301120e+00  5.14654060e-01
  1.59124004e+00  1.33346738e-01 -1.22077007e+00  1.59729110e-03
  3.22490815e-01  3.28608593e-01  3.55389819e-01 -1.55803326e-01
  1.62627416e+00 -1.34733845e+00  3.13074868e-01  3.81231677e-01
 -1.35545305e+00 -1.32465800e-01 -1.48727165e+00  1.16754801e+00
  9.27656478e-02  1.14475499e-02  9.90081363e-01  1.63565637e+00
 -8.43225170e-01 -7.55154275e-02 -1.35928082e-01 -9.79865079e-01
  1.71492580e+00  2.81221850e+00  8.08559148e-01  8.22305612e-01
 -9.99405326e-01  1.08012571e+00  1.38224340e+00 -9.13909329e-01
  6.55508963e-01 -1.28212671e+00 -7.83774540e-03  3.68443830e-01
 -8.79896054e-01  1.24975810e+00  5.94494765e-02  1.43630355e+

eval_count: 256
var_params: [-0.38254612 -0.04966069 -1.85642322  0.8967436  -1.54125616  1.34574486
  0.58545994 -1.4147555  -0.98965816  1.29806064  0.06166648  0.34224957
  2.35559564  0.08755043  0.43608016 -0.89331423 -0.29727113  0.0911658
 -1.07300186  0.46392546  1.62755643  0.03466223 -1.25125945 -0.03683275
  0.28070506  0.33200089  0.35510409 -0.13363642  1.64449573 -1.34724062
  0.2589683   0.43166462 -1.35982992 -0.09929683 -1.46732223  1.17063579
  0.10398585  0.01100663  0.9804225   1.64893312 -0.81236269 -0.11761822
 -0.12762309 -0.96513109  1.73103851  2.82799113  0.82795448  0.84022089
 -0.97936347  1.09536348  1.40229068 -0.89524937  0.67519948 -1.26342131
  0.00812979  0.38667873 -0.86906363  1.26930439  0.07396552  1.44561014
 -0.80487949 -0.01467383  0.48878791 -0.11263318 -0.51957201  1.03445042
  0.90772333  0.97121369 -1.54106175 -1.41649194  0.3644439   0.79270246]
eval_val: 0.5805835206226932
index: 256
eval_count: 257
var_params: [-3.87226444e-01 -6.54705468

eval_count: 264
var_params: [-3.99554422e-01 -4.74931114e-02 -1.85242268e+00  8.93262826e-01
 -1.53431088e+00  1.35721865e+00  5.83997479e-01 -1.41599398e+00
 -9.89684891e-01  1.29560256e+00  5.87620629e-02  3.45839951e-01
  2.36710914e+00  8.41163362e-02  4.36124405e-01 -8.96282696e-01
 -2.93967191e-01  9.70322365e-02 -1.08973887e+00  4.53631889e-01
  1.61116529e+00  2.16749580e-02 -1.25349434e+00 -5.27861240e-02
  2.71731309e-01  3.28048046e-01  3.34392125e-01 -1.42047214e-01
  1.62992251e+00 -1.35322618e+00  3.10104751e-01  4.20433757e-01
 -1.35027625e+00 -1.17725347e-01 -1.47983884e+00  1.16492240e+00
  8.78357336e-02  3.49575502e-03  9.91539618e-01  1.62255070e+00
 -8.23986720e-01 -8.65371178e-02 -1.25394849e-01 -9.32191995e-01
  1.72309824e+00  2.82034929e+00  8.38227263e-01  8.33786271e-01
 -9.94735641e-01  1.08382717e+00  1.39734625e+00 -8.94445244e-01
  6.64524126e-01 -1.27785597e+00  2.20544213e-03  3.88258458e-01
 -8.78273346e-01  1.26692292e+00  7.15147396e-02  1.42815131e+

eval_count: 272
var_params: [-0.38229211 -0.10076062 -1.89878013  0.89485032 -1.56079266  1.35977218
  0.53504245 -1.46163882 -1.00219978  1.26672666  0.07457676  0.37102947
  2.32104619  0.08750979  0.37305133 -0.94777274 -0.28532075  0.12578497
 -1.09509007  0.4899384   1.63088155  0.04444782 -1.31167733 -0.05501185
  0.2934888   0.32275878  0.32662192 -0.2079661   1.63860864 -1.36014279
  0.32405007  0.41125114 -1.27509528 -0.20968559 -1.46196812  1.11823887
  0.09807749 -0.01133655  0.92802302  1.59734792 -0.86865245 -0.13184608
 -0.05422472 -0.97898307  1.69730045  2.74386291  0.76514053  0.83760406
 -0.98077608  1.0965442   1.36754765 -0.91390578  0.66126709 -1.27781832
  0.00557623  0.36335836 -0.86798645  1.24655464  0.05056165  1.44058915
 -0.77058117 -0.01991588  0.48817442 -0.11561449 -0.41612142  1.03707927
  0.94257323  0.98902918 -1.6401655  -1.3609292   0.37335519  0.79037845]
eval_val: 0.576642138753122
index: 272
eval_count: 273
var_params: [-0.40156545 -0.11933915 -1.

eval_count: 281
var_params: [-0.39075814 -0.11126997 -1.89767534  0.96122679 -1.54019324  1.34170573
  0.59433677 -1.42874177 -0.99076058  1.28620029  0.07311432  0.3429955
  2.34741755  0.06339839  0.41637743 -0.90459837 -0.29641484  0.07143458
 -1.09018198  0.47149405  1.64704386  0.05187901 -1.32653775 -0.08536069
  0.28732354  0.36869051  0.32466562 -0.20503548  1.64191259 -1.28433975
  0.32739355  0.34483037 -1.27147542 -0.21444839 -1.46547951  1.15663481
  0.09201842 -0.0103887   0.93177822  1.60017148 -0.87389539 -0.13022291
 -0.04843386 -0.98502742  1.69926618  2.74688623  0.7761821   0.83736232
 -0.96352444  1.09653647  1.36159702 -0.90370859  0.65785519 -1.28444116
  0.00980724  0.36871287 -0.86836583  1.25189255  0.05615824  1.43888689
 -0.78728851 -0.02178087  0.49637205 -0.10994514 -0.41755579  0.99134718
  0.97834352  1.00031189 -1.63917296 -1.36337625  0.37026693  0.78452365]
eval_val: 0.5723780298414143
index: 281
eval_count: 282
var_params: [-0.39882106 -0.09639229 -1.

In [ ]:
print(result)

## Storing the optimal parameters for grading

Once the training step of the vqc algorithm is done we obtain the optimal parameters for our specific variational form. For the grading function to be able to access these optimal parameters you will need to follow the steps below. 

* **Step 1**: Run the cell below with `print(repr(vqc.optimal_params))`. 
* **Step 2**: Copy the matrix of optimal parameters and store it in the variable `optimal_parameters` inside the function `return_optimal_params()` in the next cell. This will enable us to extract it while calculating the accuracy your the model during grading. Given below is a pictographical explanation of the same:  

<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/opt_params456b075.png" width="800">

In [217]:
print(repr(vqc.optimal_params))

array([ 0.54632732,  0.40538044, -1.01295329,  1.49487678,  1.64501921,
        0.59286665,  2.12784723,  1.21054489, -0.5643853 ,  0.95213542,
       -0.27556139, -1.83490042, -0.01200562,  3.75627203,  1.43557252,
       -0.39997671, -0.5545942 ,  0.46746988, -1.27218015,  0.0318519 ,
        0.87543005, -0.25437162, -1.34021757, -2.08737164,  2.04253738,
        0.83175021,  0.77956833, -0.17427723,  2.16326879, -1.5828532 ,
        1.17114289, -1.17920076, -1.45410869, -0.89369175,  0.01718402,
       -1.59692096,  0.34737521, -2.48543596,  0.74288619,  0.28864511,
        0.57559697,  0.51204554, -0.92355601,  0.49001691, -0.6171768 ,
        0.14289195,  1.87852521,  0.44203041,  2.02773329,  1.01900935,
        1.40938457, -0.3939512 ,  0.50795543,  0.43727053,  0.64402482,
        0.43716062, -2.43465227,  0.81417097,  0.38247537, -0.64960273,
       -0.71656997,  0.64017647,  0.91821865, -0.04276097, -2.2802225 ,
       -0.90311979,  1.9641994 , -0.9093311 , -0.08099036,  0.13

In [1]:
%%write_and_run optimal_params.py
# # the write_and_run function writes the content in this cell into the file "optimal_params.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
import numpy as np
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def return_optimal_params():
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    
    optimal_parameters = [-0.39210554,-0.11272732,-1.89453842,0.96143768,-1.53968916,1.3424913
,0.59464602,-1.42870056,-0.99087096,1.28627895,0.07259188,0.34394906
,2.34826954,0.06369271,0.41586421,-0.90852375,-0.29510799,0.07195059
,-1.09032432,0.47043351,1.64920157,0.05173527,-1.32730807,-0.0866437
,0.28684457,0.36817005,0.33984315,-0.20511281,1.64259185,-1.28473659
,0.32719556,0.34459078,-1.26827538,-0.21581063,-1.46318546,1.15515741
,0.09109722,-0.01043691,0.94111106,1.59906567,-0.87413902,-0.13339905
,-0.05924921,-0.98893178,1.64697112,2.77076005,0.77032392,0.84059383
,-0.96428745,1.09433441,1.36179769,-0.90425282,0.65720088,-1.28550273
,0.00851721,0.36874422,-0.87074053,1.25241671,0.0599995,1.43953454
,-0.78834562,-0.01705501,0.49541302,-0.11104393,-0.40814493,1.04271021
,0.94777679,0.99318567,-1.64321124,-1.35563652,0.37826515,0.79001582]
    
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    return np.array(optimal_parameters)

UsageError: Cell magic `%%write_and_run` not found.


## Submission

Before we go any further, check that you have the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` in the **same working directory as this notebook**. If you do not, then go back to the start and run the notebook making sure you have filled in the code where its required. When you run the cell below, all the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` are combined into one file named **"answer.py"**. Now your working directory will have four python (.py) files out of which **"answer.py"** is the submission file: 
* `answer.py` <- upload this file onto HackerEarth and click on "Submit and Evaluate"
* `feature_map.py`
* `variational_circuit.py`
* `optimal_params.py`

In [219]:
solution = ['feature_map.py','variational_circuit.py','optimal_params.py']
file = open("answer.py","w")
file.truncate(0)
for i in solution:    
    with open(i) as f:
        with open("answer.py", "a") as f1:
            for line in f:
                f1.write(line)
file.close()

## Grading Function

Given below is the grading function that we shall use to grade your submission with a test dataset that is of the same format as `challenge_dataset_4_9.csv`. You can use it to grade your submission by extracting a few points out of the `challenge_dataset_4_9.csv` to get a basic idea of how your model is performing. 

In [220]:
#imports required for the grading function 
from qiskit import *
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit.aqua.components.variational_forms import VariationalForm
import numpy as np

### Working of the grading function

The grading function `grade()` takes as **input**: 

* `test_data`: (`np.ndarray`) -- **no. of datapoints $\times$ dimension of data** : the datapoints against which we want to test our model. 


* `test_labels`: (`np.ndarray`) -- **no. of datapoints $\times$ 1** : A column vector with each entry either 0 or 1 as entries.


* `feature_map`: (`QuantumCircuit` or `FeatureMap`) -- A quantum feature map which is the output of `feature_map()` defined earlier.


* `variational_form`: (`QuantumCircuit` or `VariationalForm`) -- A variational form which is the output of `variational_circuit()` defined earlier.


* `optimal_params`: (`numpy.ndarray`) -- the optimal parameters obtained after running the VQC algorithm above. These are the values obtained when the function `return_optimal_params()` is run. 


* `find_circuit_cost` : (`bool`) -- Calculates the circuit cost if set to `True`. Circuit cost is calculated by converting the circuit to the basis gate set `\[ 'u3', 'cx'\]` and then applying the formula **cost = 1$\times$(no.of u3 gates) + 10$\times$(no.of cx gates)**.


* `verbose` : (`bool`) -- prints the result message if set to `True`.

And gives as **output**: 

* `model_accuracy` : (`numpy.float64`) -- percent accuracy of the model. 


* `circuit_cost`: (`int`) -- circuit cost as explained above.


* `ans`: (`tuple`) -- Output of the `VQC.predict()` method. 


* `result_msg`: (`str`) -- Result message which also outputs the error message in case of one.


* `unrolled_circuit`: (`QuantumCircuit` or `None`) -- the circuit obtained after unrolling the full VQC circuit and substituting the optimal parameters to the basis gate set `\[ 'u3', 'cx'\]`.

**Note:** if you look inside the `grade()` function in Section 2 you'll see that we have initialized a COBYLA optimizer though the prediction step will not require one. Similarily we have given a dataset to `training dataset`. Both of these are dummy variables. The reason for this is because these are not optional variables the `VQC` class instantiation.  

In [221]:
def grade(test_data, test_labels, feature_map, variational_form, optimal_params, find_circuit_cost=True, verbose=True):
    seed = 10598
    model_accuracy = None 
    circuit_cost=None 
    ans = None
    unrolled_circuit = None
    result_msg=''
    data_dim = np.array(test_data).shape[1]
    dataset_size = np.array(test_data).shape[0]
    dummy_training_dataset=training_input = {'A':np.ones((2,data_dim)), 'B':np.ones((2, data_dim))}
    
    # converting 4's to 0's and 9's to 1's for checking 
    test_labels_transformed = np.where(test_labels==4, 0., 1.)
    max_qubit_count = 6
    max_circuit_cost = 2000
    
    # Section 1
    if feature_map is None:
        result_msg += 'feature_map variable is None. Please submit a valid entry' if verbose else ''
    elif variational_form is None: 
        result_msg += 'variational_form variable is None. Please submit a valid entry' if verbose else ''
    elif optimal_params is None: 
        result_msg += 'optimal_params variable is None. Please submit a valid entry' if verbose else ''
    elif test_data is None: 
        result_msg += 'test_data variable is None. Please submit a valid entry' if verbose else ''
    elif test_labels is None: 
        result_msg += 'test_labels variable is None. Please submit a valid entry' if verbose else ''
    elif not isinstance(feature_map, (QuantumCircuit, FeatureMap)):
        result_msg += 'feature_map variable should be a QuantumCircuit or a FeatureMap not (%s)' % \
                      type(feature_map) if verbose else ''
    elif not isinstance(variational_form, (QuantumCircuit, VariationalForm)):
        result_msg += 'variational_form variable should be a QuantumCircuit or a VariationalForm not (%s)' % \
                      type(variational_form) if verbose else ''
    elif not isinstance(test_data, np.ndarray):
        result_msg += 'test_data variable should be a numpy.ndarray not (%s)' % \
                      type(test_data) if verbose else ''
    elif not isinstance(test_labels, np.ndarray):
        result_msg += 'test_labels variable should be a numpy.ndarray not (%s)' % \
                      type(test_labels) if verbose else ''
    elif not isinstance(optimal_params, np.ndarray):
        result_msg += 'optimal_params variable should be a numpy.ndarray not (%s)' % \
                      type(optimal_params) if verbose else ''
    elif not dataset_size == test_labels_transformed.shape[0]:
        result_msg += 'Dataset size and label array size must be equal'
    # Section 2
    else:
        
        # setting up COBYLA optimizer as a dummy optimizer
        from qiskit.aqua.components.optimizers import COBYLA
        dummy_optimizer = COBYLA()

        # setting up the backend and creating a quantum instance
        backend = Aer.get_backend('qasm_simulator')
        backend_options = {"method": "statevector"}
        quantum_instance = QuantumInstance(backend, 
                                           shots=2000, 
                                           seed_simulator=seed, 
                                           seed_transpiler=seed, 
                                           backend_options=backend_options)

        # creating a VQC instance and running the VQC.predict method to get the accuracy of the model 
        vqc = VQC(optimizer=dummy_optimizer, 
                  feature_map=feature_map, 
                  var_form=variational_form, 
                  training_dataset=dummy_training_dataset)
        
        from qiskit.transpiler import PassManager
        from qiskit.transpiler.passes import Unroller
        pass_ = Unroller(['u3', 'cx'])
        pm = PassManager(pass_)
        # construct circuit with first datapoint
        circuit = vqc.construct_circuit(data[0], optimal_params)
        unrolled_circuit = pm.run(circuit)
        gates = unrolled_circuit.count_ops()
        if 'u3' in gates: 
            circuit_cost = gates['u3']
        if 'cx' in gates: 
            circuit_cost+= 10*gates['cx']
        
        if circuit.num_qubits > max_qubit_count:
            result_msg += 'Your quantum circuit is using more than 6 qubits. Reduce the number of qubits used and try again.'
        elif circuit_cost > max_circuit_cost:
            result_msg += 'The cost of your circuit is exceeding the maximum accpetable cost of 2000. Reduce the circuit cost and try again.'
        else: 
            
            ans = vqc.predict(test_data, quantum_instance=quantum_instance, params=np.array(optimal_params))
            model_accuracy = np.sum(np.equal(test_labels_transformed, ans[1]))/len(ans[1])

            result_msg += 'Accuracy of the model is {}'.format(model_accuracy) if verbose else ''
            result_msg += ' and circuit cost is {}'.format(circuit_cost) if verbose else ''
            
    return model_accuracy, circuit_cost, ans, result_msg, unrolled_circuit

## Process of grading using a dummy grading dataset

Let us create a dummy grading dataset with features and labels `grading_features` and `grading_labels` created from the last 2000 datapoints from `data_features` and `data_labels`so that we can a rough estimate of our accuaracy. It must be noted that this may not be a balanced dataset, i.e, may not have equal number of `4`'s and `9`'s and is not best practice. This is only given for the purpose of the demo of `grade()` function. In the final scoring done on HackerEarth, the testing dataset used will have a balanced number of class labels `4` and `9`.

In [222]:
grading_dataset_size=2000    # this value is not per digit but in total
grading_features = data_features[-grading_dataset_size:]
grading_labels = data_labels[-grading_dataset_size:]

In [223]:
start = time.process_time()

accuracy, circuit_cost, ans, result_msg, full_circuit  =  grade(test_data=grading_features, 
                                                                test_labels=grading_labels, 
                                                                feature_map=feature_map(), 
                                                                variational_form=variational_circuit(), 
                                                                optimal_params=return_optimal_params())

print("time taken: {} seconds".format(time.process_time() - start))
print(result_msg)

time taken: 169.226498146003 seconds
Accuracy of the model is 0.81 and circuit cost is 876


You can also check your **accuracy**, **circuit_cost** and **full_circuit** which is the result of combining the feature map and variational circuit and unrolling into the basis \['u3', 'cx'\].

In [52]:
print("Accuracy of the model: {}".format(accuracy))
print("Circuit Cost: {}".format(circuit_cost))
print("The complete unrolled circuit: ")
full_circuit.draw()

Accuracy of the model: 0.6835
Circuit Cost: 648
The complete unrolled circuit: 


┌───────────────┐                                                 »
q_0: ┤ U3(pi/2,0,pi) ├──────────────────────■───────────────────────■──»
     ├───────────────┤┌──────────────────┐┌─┴─┐┌─────────────────┐┌─┴─┐»
q_1: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,-0.99523) ├┤ X ├┤ U3(0,0,0.99523) ├┤ X ├»
     ├───────────────┤└┬───────────────┬─┘└───┘└─────────────────┘└───┘»
q_2: ┤ U3(pi/2,0,pi) ├─┤ U3(0,0,2.597) ├───────────────────────────────»
     └───────────────┘ └───────────────┘                               »
c_0: ══════════════════════════════════════════════════════════════════»
                                                                       »
c_1: ══════════════════════════════════════════════════════════════════»
                                                                       »
c_2: ══════════════════════════════════════════════════════════════════»
                                                                       »
«     ┌─────────────────┐                                                ┌───┐»
«q_0: ┤ U3(0,0,-0.9614) ├────────────────────────────────────────────────┤ X ├»
«     └─────────────────┘                       ┌───────────────────────┐└─┬─┘»
«q_1: ─────────■─────────────────────────────■──┤ U3(0.8228,-pi/2,pi/2) ├──┼──»
«            ┌─┴─┐       ┌────────────────┐┌─┴─┐└───────────────────────┘  │  »
«q_2: ───────┤ X ├───────┤ U3(0,0,-2.597) ├┤ X ├───────────────────────────■──»
«            └───┘       └────────────────┘└───┘                              »
«c_0: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«c_1: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«c_2: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«     ┌────────────────┐                    ┌───┐   ┌──────────────────┐   »
«q_0: ┤ U3(0,0,0.9614) ├────────────────────┤ X ├───┤ U3(4,-pi/2,pi/2) ├───»
«     ├───────────────┬┘┌──────────────────┐└─┬─┘   └──────────────────┘   »
«q_1: ┤ U3(pi/2,0,pi) ├─┤ U3(0,0,-0.99523) ├──┼────────────────────────────»
«     └───────────────┘ └──────────────────┘  │  ┌────────────────────────┐»
«q_2: ────────────────────────────────────────■──┤ U3(0.90163,-pi/2,pi/2) ├»
«                                                └────────────────────────┘»
«c_0: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_1: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_2: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«     ┌───────────────┐                                         »
«q_0: ┤ U3(pi/2,0,pi) ├────────■─────────────────────────────■──»
«     └───────────────┘      ┌─┴─┐      ┌─────────────────┐┌─┴─┐»
«q_1: ───────────────────────┤ X ├──────┤ U3(0,0,0.99523) ├┤ X ├»
«     ┌───────────────┐┌─────┴───┴─────┐└─────────────────┘└───┘»
«q_2: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,2.597) ├────────────────────────»
«     └───────────────┘└───────────────┘                        »
«c_0: ══════════════════════════════════════════════════════════»
«                                                               »
«c_1: ══════════════════════════════════════════════════════════»
«                                                               »
«c_2: ══════════════════════════════════════════════════════════»
«                                                               »
«     ┌─────────────────┐                                                ┌───┐»
«q_0: ┤ U3(0,0,-0.9614) ├────────────────────────────────────────────────┤ X 